In [1]:
import urllib.robotparser
import requests
import xml.etree.ElementTree as ET

/Users/jorgemagdaleno/PycharmProjects/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
!pip install ultimate-sitemap-parser


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [3]:
def fetch_robots_file(url, headers):
    
    # Se realiza un try y catch para hacer catch de errores por fuera del request siendo rechazado por el servidor
    try:
        
        # Se realiza un request a la url y se da un tiempo de espera de 10 segundos, 
        # en caso de que la respuesta no sea exitosa devuelve el mensaje de error
        response = requests.get(url, timeout=10, headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            print(response)
            print(f"No se pudo obtener el archivo robots.txt de {url}")
            return None
    except Exception as e:
        print(f"Error al obtener {url}: {e}")
        return None

def analyze_robots(url, content):
    
    # Por medio de robotparser tomamos el contenido y lo separamos por lineas para luego poder iterarlas
    rfp = urllib.robotparser.RobotFileParser()
    rfp.parse(content.splitlines())

    # Verificar si el User Agent puede acceder a la raíz del sitio
    can_fetch_root = rfp.can_fetch(user_agent, "/")
    print(f"El User Agent '{user_agent}' puede acceder al raíz de {url}: {can_fetch_root}")

    # Iterar sobre las reglas definidas en el archivo robots.txt
    entries = [rfp.default_entry] if rfp.default_entry else []
    entries.extend(rfp.entries)

    for entry in entries:
        for rule in entry.rulelines:
            print(f"User Agent: {entry.useragents}, Path: {rule.path}, Allowance: {rule.allowance}")


In [ ]:
# Lista de sitios a analizar
sitios = ["https://www.cicese.mx/robots.txt", 
          "https://www.uabc.mx/robots.txt", 
          "https://www.unam.mx/robots.txt", 
          "https://www.github.com/robots.txt"]

# Incluyo headers porque Unam en particular bloquea la conneccion sin ser un User Agent definido
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
}

# User Agent para revisar los permisos
user_agent = "*"

# Procesar cada sitio
for sitio in sitios:
    print(f"\nAnalizando {sitio}...")
    robots_content = fetch_robots_file(sitio, headers)
    if robots_content:
        analyze_robots(sitio, robots_content)



Analizando https://www.cicese.mx/robots.txt...
El User Agent '*' puede acceder al raíz de https://www.cicese.mx/robots.txt: True
User Agent: ['SemrushBot'], Path: /, Allowance: False
User Agent: ['SemrushBot-SA'], Path: /, Allowance: False
User Agent: ['SemrushBot-BM'], Path: /, Allowance: False
User Agent: ['SemrushBot-CT'], Path: /, Allowance: False
User Agent: ['SemrushBot-SWA'], Path: /, Allowance: False
User Agent: ['SemrushBot-SI'], Path: /, Allowance: False
User Agent: ['SemrushBot-BA'], Path: /, Allowance: False
User Agent: ['BLEXBot'], Path: /, Allowance: False

Analizando https://www.uabc.mx/robots.txt...
El User Agent '*' puede acceder al raíz de https://www.uabc.mx/robots.txt: True
User Agent: ['*'], Path: /wp-admin/, Allowance: False
User Agent: ['*'], Path: /wp-admin/admin-ajax.php, Allowance: True
User Agent: ['*'], Path: /wp-content/uploads/wpforms/, Allowance: False

Analizando https://www.unam.mx/robots.txt...


**Practica 3**

In [ ]:
def parse_wordpress_sitemap(content):
    try:
        
        # Utilizo la libreria de xml.etree.ElementTree para poder tomar el body del la pagina usarlo como xml
        root = ET.fromstring(content)
        
        # Busco los lugares donde se enlistan los otros urls de la pagina, como son extras tambien exploro estos
        namespaces = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        sitemaps = root.findall("ns:sitemap", namespaces)

        # Itero por todos buscando los sitios que refieran
        print("Sitemaps secundarios encontrados:")
        for sitemap in sitemaps:
            loc = sitemap.find("ns:loc", namespaces).text
            print(f"- {loc}")
            analyze_secondary_sitemap(loc)
    except Exception as e:
        print(f"Error al analizar el sitemap: {e}")

def analyze_secondary_sitemap(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            root = ET.fromstring(response.content)
            namespaces = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
            urls = root.findall("ns:url", namespaces)
            
            print(f"URLs encontradas en {url}:")
            for url_entry in urls:
                loc = url_entry.find("ns:loc", namespaces).text
                print(f"- {loc}")
        else:
            print(f"No se pudo acceder al sitemap secundario: {url}")
    except Exception as e:
        print(f"Error al procesar el sitemap secundario {url}: {e}")

In [ ]:
# URL del sitemap
sitemap_url = "https://www.uabc.mx/wp-sitemap.xml"

# Procesar el sitemap
sitemap_content = fetch_robots_file(sitemap_url, headers)
if sitemap_content:
    parse_wordpress_sitemap(sitemap_content)
